In [1]:
import numpy as np

import dataset

import cv2

In [2]:
HEIGHT = 64
WIDTH = 64
DEPTH = 3

DATA_PATH = '../dataset/stl10_binary/train_X.bin'
LABEL_PATH = '../dataset/stl10_binary/train_y.bin'

In [3]:
images = dataset.read_all_images(DATA_PATH)
labels = dataset.read_labels(LABEL_PATH)

In [4]:
def get_resized_images(images):
    train_X=np.empty((images.shape[0],HEIGHT,WIDTH,3),dtype=np.uint8)
    
    for i in range(images.shape[0]):
        train_X[i] = cv2.resize(images[i],(HEIGHT,WIDTH))
    
    return train_X

In [5]:
train_X = get_resized_images(images)

In [151]:
train= train_X.reshape(5000,3,64,64)

In [125]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, Deconvolution2D
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers import Activation, Dense

(64, 64)

In [180]:
def get_cnn_architecture(weights_path=None):
    model = Sequential()
    model.add(Convolution2D(64, 4, 4, activation='relu', border_mode='same', name='conv-1', input_shape = (3,64, 64)))
    model.add(Convolution2D(32, 4, 4, activation='relu', border_mode='same', name='conv-2'))
    model.add(Convolution2D(16, 4, 4, activation='relu', border_mode='same', name='conv-3'))
    model.add(Convolution2D(8, 4, 4, activation='relu', border_mode='same', name='conv-4'))
    model.add(Convolution2D(4, 4, 4, activation='relu', border_mode='same', name='conv-5'))
    
    model.add(Deconvolution2D(8, 4, 4, output_shape=(None,16,16,128) ,activation='relu', border_mode='same', name='deconv-5'))
       
    model.add(Flatten())
    model.add(Dense(1024, activation='relu' ,input_shape=(5000,1)))
    model.add(Dense(1, activation='softmax'))

    return model

In [181]:
model = get_cnn_architecture()

AttributeError: 'module' object has no attribute 'stack'

In [157]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
conv-1 (Convolution2D)           (None, 3, 64, 64)     65600       convolution2d_input_40[0][0]     
____________________________________________________________________________________________________
conv-2 (Convolution2D)           (None, 3, 64, 32)     32800       conv-1[0][0]                     
____________________________________________________________________________________________________
conv-3 (Convolution2D)           (None, 3, 64, 16)     8208        conv-2[0][0]                     
____________________________________________________________________________________________________
conv-4 (Convolution2D)           (None, 3, 64, 8)      2056        conv-3[0][0]                     
___________________________________________________________________________________________

In [158]:
model.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy'])

In [159]:
labels

array([2, 6, 2, ..., 2, 8, 6], dtype=uint8)

In [160]:
model.fit(train,labels,batch_size=32,nb_epoch=10,verbose=1,shuffle=True)

Epoch 1/10
5000/5000 [==============================] - 28s - loss: 28.5000 - acc: 0.1000    
Epoch 2/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    
Epoch 3/10
5000/5000 [==============================] - 27s - loss: 28.5000 - acc: 0.1000    
Epoch 4/10
5000/5000 [==============================] - 27s - loss: 28.5000 - acc: 0.1000    
Epoch 5/10
5000/5000 [==============================] - 25s - loss: 28.5000 - acc: 0.1000    
Epoch 6/10
5000/5000 [==============================] - 25s - loss: 28.5000 - acc: 0.1000    
Epoch 7/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    
Epoch 8/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    
Epoch 9/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    
Epoch 10/10
5000/5000 [==============================] - 26s - loss: 28.5000 - acc: 0.1000    


In [175]:
model.predict()

TypeError: Error when checking : data should be a Numpy array, or list/dict of Numpy arrays. Found: (None, array([[[146, 143, 110, ..., 123,  85, 136],
        [121,  82, 134, ...,  87, 141, 125],
        [ 89, 142, 126, ..., 162, 159, 126],
        ..., 
        [ 99,  61, 116, ...,  78, 119, 106],...